<a href="https://colab.research.google.com/github/vachkim/LECTURE/blob/master/MINING/Lec_ENGLISH_MINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 수집

### 1) 목적
- [다음](https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/#10528a4e1f25)의 기사를 크롤링한다
- BeautifulSoup를 활용해 본문 내용을 크롤링해보자

### 2) 모듈 다운
- 필요한 모듈을 다운 받는다
- requests는 url에서 데이터를 받아오기 위해 필요한 모듈이며
- beautifulsoup는 받은 데이터에서 tag를 활용, 필요한 부분을 크롤링하기 위해 사용된다

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

### 3) 텍스트 가져오기

- 크롤링 하고 싶은 url을 입력하고 html.parser를 활용해 tag를 활성화시킨다

In [ ]:
url = 'https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/#10528a4e1f25'
response = requests.get(url)
soup = bs(response.text, 'html.parser') #request한 변수에 .text하면 text만 추출된다

### 4) Tag를 통해 필요한 부분 가져오기

- 아래와 같이 find_all을 통해 p태그를 가진 본문 내용을 가져온다

In [ ]:
dl = soup.find_all('p')

- 선생님의 경우 아래와 같이 div에 클래스가 기사 내용을 가리키는 내용을 저장한 뒤
- 여기에서 다시 p태그 값을 가져와 저장

In [ ]:
#선생님 코드
div = soup.find('div', class_='article-body fs-article fs-responsive blah')
p_tag = div.find_all('p')
content = '' #빈공간의 content변수를 만든뒤
for i in p_tag: #p tag를 가진 리스트 변수의 텍스트 부분을 content에 추가
  content += i.text
content[:500]

- find_all을 사용하면 위와 같이 전체 내용이 p태그를 포함한 채 리스트 형태로 나오게 된다
- 따라서 여기에서 텍스트만 추출하고 싶으면 아래와 같이 for문을 활용, 개별적으로 text를 추출하는 것이 필요하다

In [ ]:
text = []
for line in dl:
  txt = line.text
  text.append(txt)

- 위와 같이 할 경우에는 전체 문자열이 리스트 형태로 저장된다
- 하지만 마이닝은 입력값을 문자열로 받는 경우가 많기 때문에 아래와 같이 진행하는 것이 더욱 적합하다

In [ ]:
#선생님 코드
text = ''
for line in dl:
  text += line.text
text[:500]

"Italian Renaissance: Vitruvian Man by Leonardo da VinciIt is the present-day darling of the tech world. The current renaissance of Artificial Intelligence (AI) with its sister discipline Machine Learning (ML) has led every IT firm worth its salt to engineer some form of AI onto its platform, into its toolsets and throughout its software applications.IBM CEO Ginni Rometty has already proclaimed that AI will change 100 percent of jobs over the next decade.And yes, she does mean everybody's job fro"

## 영문토큰화(Chunking)
- 자세한 내용은 [다음](https://www.nltk.org/_modules/nltk/tokenize/punkt.html) 참조

### 1) 토큰화에 사용되는 프로그램 설치
- Word Tokenize() = 마침표와 구두점(온점, 쉼표, 물음표 등)으로 구분해 문자열을 토큰화
- 토큰화에 사용되는 프로그램을 설치한다

In [ ]:
!pip install nltk

- 설치한 프로그램을 import한다
- 영문 토큰화를 위해서는 nltk.download('punkt')도 함께 입력해주어야한다

In [ ]:
import nltk
nltk.download('punkt') #tokenizer를 사용하는 기준데이터이다
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 2) 기초토큰화
- Word_Tokenizer를 활용해 앞서 불러온 text파일은 구두점을 기준으로 나눈다

In [ ]:
token1 = word_tokenize(text)
print(token1[:20])

['Italian', 'Renaissance', ':', 'Vitruvian', 'Man', 'by', 'Leonardo', 'da', 'VinciIt', 'is', 'the', 'present-day', 'darling', 'of', 'the', 'tech', 'world', '.', 'The', 'current']


- WordPunctTokenizer를 사용하면 알파벳과 알파벳이 아닌 문자를 구분한다
- 'present-day'로 반환되는 위의 경우와 달리 아래는 'present''-''day'로 문자열을 구분한다

In [ ]:
import nltk
from nltk.tokenize import WordPunctTokenizer
token2 = WordPunctTokenizer().tokenize(text)
print(token2[:20])

['Italian', 'Renaissance', ':', 'Vitruvian', 'Man', 'by', 'Leonardo', 'da', 'VinciIt', 'is', 'the', 'present', '-', 'day', 'darling', 'of', 'the', 'tech', 'world', '.']


- TreebankWordTokenizer는 정규표현식에 기반한 토큰화를 진행한다

In [ ]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
token3 = TreebankWordTokenizer().tokenize(text)
print(token3[:20])

['Italian', 'Renaissance', ':', 'Vitruvian', 'Man', 'by', 'Leonardo', 'da', 'VinciIt', 'is', 'the', 'present-day', 'darling', 'of', 'the', 'tech', 'world.', 'The', 'current', 'renaissance']


## 품사부착(Pos-Tag)
- 분리한 토큰에 품사를 부착한다
- 자세한 내용은 [다음](http://www.nltk.org/api/nltk.tag.html)을 참조한다

### 1) 품사부착에 사용되는 프로그램 설치

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger') #품사 부착을 위한 기준 데이터 셋

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### 2) 품사부착 
- pos_tag기능을 활용해 토큰별로 품사를 확인한다

In [ ]:
taggedToken = pos_tag(token1)
print(taggedToken[:20])

[('Italian', 'JJ'), ('Renaissance', 'NNP'), (':', ':'), ('Vitruvian', 'JJ'), ('Man', 'NN'), ('by', 'IN'), ('Leonardo', 'NNP'), ('da', 'NN'), ('VinciIt', 'NNP'), ('is', 'VBZ'), ('the', 'DT'), ('present-day', 'JJ'), ('darling', 'NN'), ('of', 'IN'), ('the', 'DT'), ('tech', 'JJ'), ('world', 'NN'), ('.', '.'), ('The', 'DT'), ('current', 'JJ')]


- Tag로 사용된 품사 약칭은 [다음](https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/)을 참조해 확인한다

## 개체명 인식(Ne-Chunk)

### 1) 개체명 인식에 사용되는 라이브러리 설치

In [ ]:
nltk.download('words')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

### 2) 개체명 인식 진행
- 단어구분과 품사 부착, 개체명 인식 순으로 마이닝을 진행한다

In [ ]:
import nltk 
nltk.download('punkt')
from nltk.tokenize import word_tokenize

#토큰화
token = word_tokenize('Barack Obama likes fried chicken very much')
print('token=', token)

#품사부착
taggedToken = pos_tag(token)
print('pos-tag=', taggedToken)

#Ne-Chunk를 통한 개체명 인식
from nltk import ne_chunk
neToken = ne_chunk(taggedToken)
print(neToken)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
token= ['Barack', 'Obama', 'likes', 'fried', 'chicken', 'very', 'much']
pos-tag= [('Barack', 'NNP'), ('Obama', 'NNP'), ('likes', 'VBZ'), ('fried', 'VBN'), ('chicken', 'JJ'), ('very', 'RB'), ('much', 'JJ')]
(S
  (PERSON Barack/NNP)
  (ORGANIZATION Obama/NNP)
  likes/VBZ
  fried/VBN
  chicken/JJ
  very/RB
  much/JJ)
